In [1]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 52.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, value
from datetime import datetime, timedelta

# Load the person and task data
person_data = pd.read_excel('/content/Employee_table.xlsx', sheet_name="person")
df1 = pd.DataFrame(person_data)

task_data = pd.read_excel('/content/Employee_table.xlsx', sheet_name="tasks")
df2 = pd.DataFrame(task_data)

In [5]:
import time

start_time = time.time()

# Create skill_to_employee dictionary with efficiency
skill_to_employee = {}
employee_efficiency = {}
for _, row in df1.iterrows():
    employee = row['name']
    skill = row['skills']
    efficiency = row['efficiency']
    if skill not in skill_to_employee:
        skill_to_employee[skill] = []
    skill_to_employee[skill].append(employee)

    # Store efficiency for each employee
    if employee not in employee_efficiency:
        employee_efficiency[employee] = efficiency

# Calculate target hours (adjusted for efficiency)
total_task_duration = df2['duration'].sum()
total_efficiency = sum(employee_efficiency.values())
target_hours = total_task_duration / total_efficiency  # Ideal workload per efficiency unit

# Initialize optimization problem
prob = LpProblem("Task_Assignment_Balanced", LpMinimize)

# Decision variables
task_assignment = {}
for _, task_row in df2.iterrows():
    task_name = task_row['task_name']
    required_skill = task_row['required_skills']
    duration = task_row['duration']

    # Only consider employees with the required skill for each task
    if required_skill in skill_to_employee:
        for employee in skill_to_employee[required_skill]:
            task_assignment[(employee, task_name)] = LpVariable(f"x_{employee}_{task_name}", 0, 1, LpBinary)

# Deviation variables to balance workload
deviation_above = LpVariable("Deviation_Above", lowBound=0)
deviation_below = LpVariable("Deviation_Below", lowBound=0)

# Objective function: Minimize workload deviation
prob += deviation_above + deviation_below, "Minimize_workload_deviation"

# Constraint 1: Each task must be assigned to exactly one employee with the matching skill
for _, task_row in df2.iterrows():
    task_name = task_row['task_name']
    required_skill = task_row['required_skills']

    # Only proceed if the skill has matching employees
    if required_skill in skill_to_employee:
        prob += lpSum([task_assignment[(employee, task_name)] for employee in skill_to_employee[required_skill]]) == 1, f"Task_Assignment_{task_name}"
    else:
        print(f"Task {task_name} requires skill {required_skill} but no employee has it.")

# Constraint 2: Ensure each employee's workload stays within target hours and minimize deviations
for employee in df1['name'].unique():
    # Total hours for each employee, adjusted for efficiency
    employee_total_hours = lpSum([task_assignment[(employee, task_name)] * df2.loc[df2['task_name'] == task_name, 'duration'].values[0]
                                  for task_name in df2['task_name'] if (employee, task_name) in task_assignment])

    # Adjust target hours based on employee's efficiency
    employee_target = target_hours * employee_efficiency[employee]

    # Ensure workload is close to target with allowed deviation
    prob += employee_total_hours <= employee_target + deviation_above, f"Max_Hours_{employee}"
    prob += employee_total_hours >= employee_target - deviation_below, f"Min_Hours_{employee}"

# Solve the optimization problem
prob.solve()

# Extract and format the results
output_data = []
start_date = datetime.strptime("16-12-2024", "%d-%m-%Y")  # Start date for Day-1

for employee in df1['name'].unique():
    assigned_tasks = []
    required_skills = []
    total_hours = 0
    daily_distribution = {}
    current_hour = 1
    daily_limit = 8  # Maximum hours per day

    for _, task_row in df2.iterrows():
        task_name = task_row['task_name']
        duration = task_row['duration']
        required_skill = task_row['required_skills']

        # Check if task is assigned to the employee
        if (employee, task_name) in task_assignment and task_assignment[(employee, task_name)].varValue == 1:
            assigned_tasks.append(task_name)
            required_skills.append(required_skill)
            total_hours += duration

            # Assign tasks day-by-day with carry-over logic based on 8-hour workday
            hours_remaining = duration
            while hours_remaining > 0:
                day = (current_hour - 1) // daily_limit + 1
                if day not in daily_distribution:
                    daily_distribution[day] = []

                # Calculate available hours left in the current day
                hours_in_day = daily_limit - ((current_hour - 1) % daily_limit)

                if hours_remaining <= hours_in_day:
                    # Task can be completed within the current day's hours
                    daily_distribution[day].append(f"{task_name} ({hours_remaining} hours)")
                    current_hour += hours_remaining
                    hours_remaining = 0
                else:
                    # Part of the task will be completed today, remaining will go to next day
                    daily_distribution[day].append(f"{task_name} ({hours_in_day} hours)")
                    hours_remaining -= hours_in_day
                    current_hour += hours_in_day

    # Calculate days and remaining hours
    days = total_hours // daily_limit
    remaining_hours = total_hours % daily_limit

    # Prepare day-by-day utilization columns with actual dates
    max_day = days + (1 if remaining_hours > 0 else 0)
    day_utilization = {}
    for day in range(1, max_day + 1):
        current_date = start_date + timedelta(days=day - 1)
        day_utilization[f'{current_date.strftime("%d-%m-%Y")}'] = ', '.join(daily_distribution.get(day, ['available']))

    # If max days < total days available, fill the remaining days as 'available'
    for day in range(max_day + 1, 6):  # Assuming 5-day workweek
        current_date = start_date + timedelta(days=day - 1)
        day_utilization[f'{current_date.strftime("%d-%m-%Y")}'] = 'available'

    # Add to output if tasks were assigned
    if assigned_tasks:
        output_data.append({
            'Employee': employee,
            'Assigned_Tasks': ', '.join(assigned_tasks),
            'Required_Skills': ', '.join(required_skills),
            'Total_Hours': total_hours,
            'Days': f"{days} days, {remaining_hours} hours",
            **day_utilization
        })

# Calculate total project duration
total_project_hours = max(d['Total_Hours'] for d in output_data)

# Calculate contribution for each employee
for d in output_data:
    d['Person_Contribution (%)'] = (d['Total_Hours'] / total_project_hours) * 100

# Capture unassigned tasks for NA employees
unassigned_tasks = [task['task_name'] for task in df2.to_dict('records') if task['task_name'] not in [t for d in output_data for t in d['Assigned_Tasks'].split(', ')]]
for task in unassigned_tasks:
    output_data.append({
        'Employee': 'NA',
        'Assigned_Tasks': task,
        'Required_Skills': df2.loc[df2['task_name'] == task, 'required_skills'].values[0],
        'Total_Hours': df2.loc[df2['task_name'] == task, 'duration'].values[0],
        'Days': 'NA',
        'Person_Contribution (%)': 'NA',
        **{f'{(start_date + timedelta(days=day - 1)).strftime("%d-%m-%Y")}': 'NA' for day in range(1, 6)}
    })

# Display output
output_df = pd.DataFrame(output_data)
l = len(output_df['Employee'].dropna().unique()) -1
assigned_tasks = output_df.iloc[:l]

unassigned_tasks_df = output_df[output_df['Employee'] == 'NA'][['Assigned_Tasks', 'Required_Skills', 'Total_Hours']]

# Save to CSV files
assigned_tasks.to_csv("pulp_assigned_tasks.csv", index=False)
unassigned_tasks_df.to_csv("pulp_unassigned_tasks.csv", index=False)

end_time = time.time()
print(f"Execution Time: {end_time - start_time} seconds")


Task task2d requires skill nodejs but no employee has it.
Task task12b requires skill go but no employee has it.
Task task13a requires skill nodejs but no employee has it.
Task task13b requires skill go but no employee has it.
Task task15a requires skill go but no employee has it.
Task task16a requires skill go but no employee has it.
Task task17b requires skill go but no employee has it.
Task task18b requires skill go but no employee has it.
Execution Time: 0.15888547897338867 seconds
